In [1]:
#Import packages

import pandas as pd
import numpy as np
from datetime import timedelta
from scipy.fftpack import fft, ifft,rfft
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load

In [2]:
#Read data
test_data = pd.read_csv('test.csv',header=None)

In [3]:
#Create Feature Matrix function
def create_meal_feature_matrix(meal_data):
    pow_first_max =[]
    pow_second_max =[]
    pow_third_max =[]
    ind_third_max=[]
    ind_second_max =[]
    meal_data_temp = meal_data.interpolate(method = 'linear',axis = 1)
    #indices_to_drop = meal_data_temp.isna().sum(axis=1).replace(0,np.nan).dropna().index 
    #cleaned_data = meal_data_temp.drop(meal_data.index[indices_to_drop])
    cleaned_data_meal = meal_data_temp.dropna().reset_index()
    for i in range(len(cleaned_data_meal)):
        arr = abs(rfft(cleaned_data_meal.iloc[:,0:30].iloc[i].values.tolist())).tolist()
        arr_copy = arr.copy()
        arr_copy.sort()
        pow_first_max.append(arr_copy[-2])
        pow_second_max.append(arr_copy[-3])
        pow_third_max.append(arr_copy[-4])
        ind_second_max.append(arr.index(arr_copy[-3]))
        ind_third_max.append(arr.index(arr_copy[-4]))
    meal_feat_matrix = pd.DataFrame()
    meal_feat_matrix['tau_time']=(cleaned_data_meal.iloc[:,22:25].idxmin(axis=1)-cleaned_data_meal.iloc[:,5:19].idxmax(axis=1))*5
    meal_feat_matrix['CGM_diff_normalized']=(cleaned_data_meal.iloc[:,5:19].max(axis=1)-cleaned_data_meal.iloc[:,22:25].min(axis=1))/(cleaned_data_meal.iloc[:,22:25].min(axis=1))
    meal_feat_matrix['pow_second_max']=pow_second_max
    meal_feat_matrix['ind_second_max']=ind_second_max
    meal_feat_matrix['pow_third_max']=pow_third_max
    meal_feat_matrix['ind_third_max']=ind_third_max
    time = cleaned_data_meal.iloc[:,22:25].idxmin(axis = 1)
    maximum = cleaned_data_meal.iloc[:,5:19].idxmax(axis = 1)
    temp_list = []
    sec_diff_data = []
    std_dev = []
    for i in range(len(cleaned_data_meal)):
        temp_list.append(np.diff(cleaned_data_meal.iloc[:,maximum[i]:time[i]].iloc[i].tolist()).max())
        sec_diff_data.append(np.diff(np.diff(cleaned_data_meal.iloc[:,maximum[i]:time[i]].iloc[i].tolist())).max())
        std_dev.append(np.std(cleaned_data_meal.iloc[i]))
    meal_feat_matrix['sec_differential']=sec_diff_data
    meal_feat_matrix['std_deviation']=std_dev
    #meal_feat_matrix['label']=1
    return meal_feat_matrix


In [4]:
#Call function to get feature matrix of dataset
dataset = create_meal_feature_matrix(test_data)
dataset.columns = [0,1,2,3,4,5,6,7]
dataset


,0,1,2,3,4,5,6,7
0,20,0.052632,159.684583,3,141.248786,11,-1.0,35.437065
1,75,0.102362,165.497397,13,162.215522,15,21.0,28.389125
2,85,0.695652,221.785241,4,218.590170,9,8.0,58.652277
3,60,0.309091,176.264651,4,168.391708,2,6.0,32.414170
4,35,0.086758,574.681960,1,280.910578,4,12.0,63.596176
...,...,...,...,...,...,...,...,...
222,90,0.423529,254.822863,1,225.996963,3,9.0,33.525012
223,20,-0.109375,197.113346,2,145.992454,7,-1.0,28.896948
224,45,0.132450,83.308215,17,79.639817,23,1.0,14.841348
225,45,0.147059,90.568958,7,83.809017,9,1.0,16.765011


In [5]:
#predict
with open('DecisionTreeClassifier.pickle', 'rb') as pre_trained:
    pickle_file = load(pre_trained)
    predict = pickle_file.predict(dataset)    
    pre_trained.close()

In [6]:
#Store results
pd.DataFrame(predict).to_csv('Result2.csv',index=False,header=False)

In [99]:
#dataset

,tau_time,CGM_diff_normalized,pow_second_max,ind_second_max,pow_third_max,ind_third_max,sec_differential,std_deviation,label
0,20,0.052632,159.684583,3,141.248786,11,-1.0,35.437065,1
1,75,0.102362,165.497397,13,162.215522,15,21.0,28.389125,1
2,85,0.695652,221.785241,4,218.590170,9,8.0,58.652277,1
3,60,0.309091,176.264651,4,168.391708,2,6.0,32.414170,1
4,35,0.086758,574.681960,1,280.910578,4,12.0,63.596176,1
...,...,...,...,...,...,...,...,...,...
222,90,0.423529,254.822863,1,225.996963,3,9.0,33.525012,1
223,20,-0.109375,197.113346,2,145.992454,7,-1.0,28.896948,1
224,45,0.132450,83.308215,17,79.639817,23,1.0,14.841348,1
225,45,0.147059,90.568958,7,83.809017,9,1.0,16.765011,1


In [77]:
#pd.DataFrame(predict[50:150])

,0
0,1
1,1
2,1
3,1
4,1
...,...
95,0
96,0
97,1
98,0
